In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# os.environ['http_proxy'] = "http://127.0.0.1:3128"
# os.environ['https_proxy'] = "http://127.0.0.1:3128"

In [2]:
import matplotlib.pyplot as plt
from torch.utils.data import Dataset

import numpy as np
import pandas as pd
import os
from sklearn.metrics import  precision_recall_fscore_support, accuracy_score
from sklearn import preprocessing

In [3]:
from torch.utils.tensorboard import SummaryWriter

In [4]:
import torch

In [5]:
import transformers

In [6]:
exclude_user = [10]
exclude_demo_user = [310]

In [7]:
train_paths_X = []
train_excl_paths_X = []
test_paths_X = []
test_excl_paths_X = []
test_excl_demo_paths_X = []

train_paths_Y = []
train_excl_paths_Y = []
test_paths_Y = []
test_excl_paths_Y = []
test_excl_demo_paths_Y = []

In [8]:
all_paths = ['/home/data/HSE_math_exp_2/processing_internal.v2/sliced.limited_ch/' + j for j in os.listdir('/home/data/HSE_math_exp_2/processing_internal.v2/sliced.limited_ch') if '.npy' in j and 'y.npy' not in j]

In [9]:
[i for i in all_paths if 'x_74_' in i]

['/home/data/HSE_math_exp_2/processing_internal.v2/sliced.limited_ch/x_74_4.npy',
 '/home/data/HSE_math_exp_2/processing_internal.v2/sliced.limited_ch/x_74_61.npy',
 '/home/data/HSE_math_exp_2/processing_internal.v2/sliced.limited_ch/x_74_18.npy',
 '/home/data/HSE_math_exp_2/processing_internal.v2/sliced.limited_ch/x_74_46.npy',
 '/home/data/HSE_math_exp_2/processing_internal.v2/sliced.limited_ch/x_74_5.npy',
 '/home/data/HSE_math_exp_2/processing_internal.v2/sliced.limited_ch/x_74_9.npy',
 '/home/data/HSE_math_exp_2/processing_internal.v2/sliced.limited_ch/x_74_39.npy',
 '/home/data/HSE_math_exp_2/processing_internal.v2/sliced.limited_ch/x_74_10.npy',
 '/home/data/HSE_math_exp_2/processing_internal.v2/sliced.limited_ch/x_74_28.npy',
 '/home/data/HSE_math_exp_2/processing_internal.v2/sliced.limited_ch/x_74_59.npy',
 '/home/data/HSE_math_exp_2/processing_internal.v2/sliced.limited_ch/x_74_1.npy',
 '/home/data/HSE_math_exp_2/processing_internal.v2/sliced.limited_ch/x_74_54.npy',
 '/home/

In [10]:
for user in range(23, 104):
    user_paths_x = [i for i in all_paths if int(i.split('/')[-1].split('_')[1]) == user]
    user_paths_x = sorted(user_paths_x, key=lambda x: int(x.split('/')[-1].split('_')[2].split('.')[0]))
    user_paths_y = ['/home/data/HSE_math_exp_2/processing_internal.v2/sliced.limited_ch/{}_y.npy'.format(user)] * len(user_paths_x)

    user_paths = list(zip(user_paths_x, user_paths_y))

    length = len(user_paths)
    if user in exclude_user:
        continue
    #     train_excl_paths_X.extend(user_paths[:int(length * 0.66)])
    #     test_excl_paths_X.extend(user_paths[int(length * 0.66):])
    # else:
    else:
        t1 = user_paths[:int(length * 0.66)][::3]
        train_paths_X.extend(t1)
        test_paths_X.extend(user_paths[int(length * 0.66):])
        
for user in exclude_user:
    user_paths_x = [i for i in all_paths if int(i.split('/')[-1].split('_')[1]) == user]
    user_paths_x = sorted(user_paths_x, key=lambda x: int(x.split('/')[-1].split('_')[2].split('.')[0]))
    user_paths_y = ['/home/data/HSE_math_exp_2/processing_internal.v2/sliced.limited_ch/{}_y.npy'.format(user)] * len(user_paths_x)

    user_paths = list(zip(user_paths_x, user_paths_y))

    length = len(user_paths)
    train_excl_paths_X.extend(user_paths[:int(length * 0.5)])
    test_excl_paths_X.extend(user_paths[int(length * 0.5):])
    
for user in exclude_demo_user:
    user_paths_x = [i for i in all_paths if int(i.split('/')[-1].split('_')[1]) == user]
    user_paths_x = sorted(user_paths_x, key=lambda x: int(x.split('/')[-1].split('_')[2].split('.')[0]))
    user_paths_y = ['/home/data/HSE_math_exp_2/processing_internal.v2/sliced.limited_ch/{}_y.npy'.format(user)] * len(user_paths_x)

    user_paths = list(zip(user_paths_x, user_paths_y))

    length = len(user_paths)
    test_excl_demo_paths_X.extend(user_paths)

In [11]:
len(train_excl_paths_X), len(test_excl_paths_X)

(122, 122)

In [12]:
len(train_paths_X), len(test_paths_X)

(19469, 30115)

In [13]:
np.load(train_excl_paths_X[0][0]).shape

(1500, 15)

In [14]:
channels = ['time', 'Fp1', 'Fz', 'F3', 'F7', 'HEOG', 'FC5', 'FC1', 'C3', 'T7',
        'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'CP6',
        'CP2', 'Cz', 'C4', 'T8', 'VEOG', 'FC6', 'FC2', 'F4', 'F8', 'Fp2',
        'Photosensor', 'skinconductance', 'heartrate', 'respiratory', 'Sync',
        'zygomaticus', 'corrugator']

In [15]:
len(channels)

38

In [16]:
channels = ['Fp1', 'FP2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'C3', 'Cz', 'C4', \
            'P3', 'Pz', 'P4', 'O1', 'O2']

In [17]:
channels_selected_indexes = [channels.index(i) for i in channels]

In [18]:
class TEST(torch.utils.data.Dataset):
    def __init__(self, paths):
        super(TEST, self).__init__()
        self.paths = paths
#         for p in self.paths[:5]:
#             data = np.load(p[0])
#             self.data.extend(data)
#             slice_num = int(p[0].split('/')[-1].split('_')[2].split('.')[0])
            
#             label = np.load(p[1])
        
#             self.labels.append(label[slice_num])
        
            # s['data'] = np.clip(preprocessing.scale(s['data'][:, channels_selected_indexes]), -3, 3)
        # self.paths = ['{}/{}'.format(self.main_path, i) for i in os.listdir(self.main_path)]

    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, idx):
        path = self.paths[idx]
        data = np.load(path[0])
        data = np.clip(preprocessing.scale(data[:, channels_selected_indexes]), -3, 3)
        slice_num = int(path[0].split('/')[-1].split('_')[2].split('.')[0])

        label = np.load(path[1])[slice_num]
        
        return {
            'eeg': torch.from_numpy(data).float().T,
            'label': torch.tensor(label).long()
        }

In [19]:
train_dataset = TEST(train_paths_X)
test_dataset = TEST(test_paths_X)

train_excl_dataset = TEST(train_excl_paths_X)
test_excl_dataset = TEST(test_excl_paths_X)

In [20]:
class ResBlock(torch.nn.Module):
    def __init__(self, input_filters, n_filters):
        super().__init__()
        self.model1 = torch.nn.Sequential(
            torch.nn.Conv1d(input_filters, n_filters, 3, 1, padding='same'),
            torch.nn.BatchNorm1d(n_filters),
            torch.nn.ReLU(),
            
            torch.nn.Conv1d(n_filters, n_filters, 3, padding='same'),
            torch.nn.BatchNorm1d(n_filters),
            torch.nn.ReLU()
        )
        
        self.model2 = torch.nn.Sequential(
            torch.nn.Conv1d(input_filters, n_filters, 3, 1, padding='same'),
            torch.nn.BatchNorm1d(n_filters),
            torch.nn.ReLU(),
        )
        
    def forward(self, x):
        p = self.model1(x)
        o = self.model2(x)
        
        o = p + o
        o = torch.nn.functional.relu(o)
        
        return o

In [21]:
class IdenBlock(torch.nn.Module):
    def __init__(self, input_filters, n_filters):
        super().__init__()
        self.model = torch.nn.Sequential(
            torch.nn.Conv1d(input_filters, n_filters, 3, 1, padding='same'),
            torch.nn.BatchNorm1d(n_filters),
            torch.nn.ReLU(),
            
            torch.nn.Conv1d(n_filters, n_filters, 3, padding='same'),
            torch.nn.BatchNorm1d(n_filters)
        )
        
    def forward(self, x):
        p = self.model(x)
        
        o = p + x
        o = torch.nn.functional.relu(o)
        
        return o

In [22]:
class Embedder(torch.nn.Module):
    def __init__(self, length):
        super().__init__()
        self.length = length
        
        self.embedding = torch.nn.Sequential(
            torch.nn.Conv1d(15, 64, 7, padding='same'),
            torch.nn.BatchNorm1d(64),
            torch.nn.ReLU(),
            torch.nn.MaxPool1d(2),
            
            IdenBlock(64, 64),
            IdenBlock(64, 64),
            
            ResBlock(64, 128),
            ResBlock(128, 128),
            
            # ResBlock(128, 256),
            # ResBlock(256, 256),
        )
        
        self.rnn_block1 = torch.nn.Sequential(
            torch.nn.LSTM(128, 80, 1, bidirectional=True, batch_first=True)
        )
        self.norm = torch.nn.BatchNorm1d(80)
        self.rnn_block2 = torch.nn.Sequential(
            torch.nn.LSTM(160, 32, 1, bidirectional=True, batch_first=True)
        )
        
        # self.output = torch.nn.Sequential(
        #     torch.nn.Linear(64, 3),
        #     torch.nn.Softmax(-1)
        # )
        
    def forward(self, x):
        x = self.embedding(x)
        x = x.transpose(1, 2)
        
        x = x[:, ::10]
        
        x, _ = self.rnn_block1(x)
        x = torch.nn.functional.dropout1d(x, 0.1)
        _, (x, _) = self.rnn_block2(x)
        x = x.transpose(0, 1)
        x = x.reshape(x.shape[0], -1)
        
        return x

In [23]:
class RESLSTM(torch.nn.Module):
    def __init__(self, length):
        super().__init__()
        self.length = length
        
        self.embedding = Embedder(length)
        
        self.output = torch.nn.Sequential(
            torch.nn.Linear(64, 3),
            torch.nn.Softmax(-1)
        )
        
    def forward(self, x):
        x = self.embedding(x)
        x = self.output(x)
        
        return x

In [24]:
model = RESLSTM(200)

In [25]:
model = model.cuda()

In [26]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, weight_decay=0.00001)

In [27]:
loss_fnc = torch.nn.CrossEntropyLoss()

In [28]:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=32, 
    shuffle=True,
    num_workers=1
)
test_dataloader = torch.utils.data.DataLoader(
    test_dataset, 
    batch_size=32, 
    shuffle=True,
    num_workers=1
)

train_excl_dataloader = torch.utils.data.DataLoader(
    train_excl_dataset, 
    batch_size=32, 
    shuffle=True,
    num_workers=1
)
test_excl_dataloader = torch.utils.data.DataLoader(
    test_excl_dataset, 
    batch_size=32, 
    shuffle=True,
    num_workers=1
)


In [29]:
for epoch in range(25):
    print('Epoch: ', epoch)
    for batch in train_dataloader:
        optimizer.zero_grad()
        predict = model(batch['eeg'].cuda())
        
        loss = loss_fnc(predict, batch['label'].cuda())
        loss.backward()
        
        optimizer.step()
    
    test_predicts = []
    test_reals = []
    for test_batch in test_dataloader:
        with torch.no_grad():
            predict = model(test_batch['eeg'].cuda())
            test_predicts.extend(predict.cpu().tolist())
            test_reals.extend(test_batch['label'].tolist())
            
    test_predicts = np.array(test_predicts)
    test_reals = np.array(test_reals)
    
    print('Test scores')
    print(precision_recall_fscore_support(test_reals, test_predicts.argmax(-1)))
    
    val_predicts = []
    val_reals = []
    for val_batch in test_excl_dataloader:
        with torch.no_grad():
            predict = model(val_batch['eeg'].cuda())
            val_predicts.extend(predict.cpu().tolist())
            val_reals.extend(val_batch['label'].tolist())
            
    val_predicts = np.array(val_predicts)
    val_reals = np.array(val_reals)
    
    print('Val scores')
    print(precision_recall_fscore_support(val_reals, val_predicts.argmax(-1)))

Epoch:  0
Test scores
(array([0.43879509, 0.35859207, 0.37141925]), array([0.23565173, 0.22933255, 0.66748071]), array([0.30663002, 0.27975298, 0.47726479]), array([ 9705, 10173, 10237]))
Val scores
(array([0.45454545, 0.34375   , 0.44303797]), array([0.25      , 0.22      , 0.67307692]), array([0.32258065, 0.26829268, 0.53435115]), array([20, 50, 52]))
Epoch:  1
Test scores
(array([0.43905773, 0.38621746, 0.38671357]), array([0.43019062, 0.12395557, 0.65507473]), array([0.43457895, 0.18767674, 0.48632968]), array([ 9705, 10173, 10237]))
Val scores
(array([0.24137931, 0.35714286, 0.46835443]), array([0.35      , 0.1       , 0.71153846]), array([0.28571429, 0.15625   , 0.5648855 ]), array([20, 50, 52]))
Epoch:  2
Test scores
(array([0.44551363, 0.41282443, 0.41519376]), array([0.54384338, 0.13290082, 0.60808831]), array([0.48979213, 0.20107079, 0.49346017]), array([ 9705, 10173, 10237]))
Val scores
(array([0.21621622, 0.35      , 0.44615385]), array([0.4       , 0.14      , 0.55769231])

In [30]:
print(accuracy_score(test_reals, test_predicts.argmax(-1)))

0.5103104765067242


In [31]:
precision_recall_fscore_support(test_reals, test_predicts.argmax(-1))

(array([0.51571483, 0.46418366, 0.55351522]),
 array([0.55455951, 0.46308857, 0.51528768]),
 array([0.53443225, 0.46363547, 0.53371781]),
 array([ 9705, 10173, 10237]))

In [32]:
class TEST(torch.utils.data.Dataset):
    def __init__(self, paths):
        super(TEST, self).__init__()
        self.paths = paths

    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, idx):
        path = self.paths[idx]
        slice_num = int(path[0].split('/')[-1].split('_')[2].split('.')[0])
        user = int(path[0].split('/')[-1].split('_')[1])
        prev_paths = ['/home/data/HSE_math_exp_2/processing_internal.v2/sliced.limited_ch/x_{}_{}.npy'.format(user, i) for 
                      i in range(slice_num-5, slice_num+1)]
        
        data = [np.load(p) for p in prev_paths]
        data = [np.clip(preprocessing.scale(d[:, channels_selected_indexes]), -3, 3) for d in data]
        data = np.array(data)

        label = np.load(path[1])[slice_num]
        
        return {
            'eeg': torch.from_numpy(data).float().T,
            'label': (torch.tensor(label).long() > 0).float()
        }

In [33]:
train_dataset_F = TEST([i for i in train_paths_X if int(i[0].split('/')[-1].split('_')[2].split('.')[0]) >= 5])
test_dataset_F = TEST([i for i in test_paths_X if int(i[0].split('/')[-1].split('_')[2].split('.')[0]) >= 5])

train_excl_dataset_F = TEST([i for i in train_excl_paths_X if int(i[0].split('/')[-1].split('_')[2].split('.')[0]) >= 5])
test_excl_dataset_F = TEST([i for i in test_excl_paths_X if int(i[0].split('/')[-1].split('_')[2].split('.')[0]) >= 5])
                            
test_excl_demo_dataset_F = TEST([i for i in test_excl_demo_paths_X if int(i[0].split('/')[-1].split('_')[2].split('.')[0]) >= 5])

In [34]:
train_dataloader_F = torch.utils.data.DataLoader(
    train_dataset_F, 
    batch_size=32, 
    shuffle=True,
    num_workers=1
)
test_dataloader_F = torch.utils.data.DataLoader(
    test_dataset_F, 
    batch_size=32, 
    shuffle=False,
    num_workers=1
)

train_excl_dataloader_F = torch.utils.data.DataLoader(
    train_excl_dataset_F, 
    batch_size=32, 
    shuffle=True,
    num_workers=1
)
test_excl_dataloader_F = torch.utils.data.DataLoader(
    test_excl_dataset_F, 
    batch_size=32, 
    shuffle=False,
    num_workers=1
)
test_excl_demo_dataloader_F = torch.utils.data.DataLoader(
    test_excl_demo_dataset_F, 
    batch_size=32, 
    shuffle=False,
    num_workers=1
)


In [35]:
class SecondSimpleModel(torch.nn.Module):
    def __init__(self, embedder):
        super().__init__()
        
        self.embedder = embedder
        
        self.model = torch.nn.Sequential(
            torch.nn.Conv1d(64, 128, 1, padding='same'),
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU()
        )
        self.rnn = torch.nn.LSTM(128, 64, batch_first=True)
        self.predict = torch.nn.Sequential(
            torch.nn.Linear(64, 1)
        )
        
    def forward(self, x):
        x = [self.embedder(x[:, :, :, i])[:, :, None] for i in range(6)]
        x = torch.cat(x, -1)
        
        x = self.model(x)
        x = x.transpose(1, 2)
        _, (x, _) = self.rnn(x)
        x = x.transpose(0, 1)
        x = self.predict(x)
        
        return x

In [36]:
model2 = SecondSimpleModel(model.embedding).cuda()

In [37]:
optimizer2 = torch.optim.AdamW(model2.parameters(), lr=1e-5, weight_decay=0.00001)

In [38]:
loss_fnc2 = torch.nn.BCEWithLogitsLoss()

In [ ]:
for epoch in range(7):
    print('Epoch: ', epoch)
    for batch in train_dataloader_F:
        optimizer2.zero_grad()
        predict = model2(batch['eeg'].cuda()).view(-1)
        
        loss = loss_fnc(predict, batch['label'].cuda())
        loss.backward()
        
        optimizer2.step()
    
    if epoch % 5 == 0:
        test_predicts = []
        test_reals = []
        for test_batch in test_dataloader_F:
            with torch.no_grad():
                predict = model2(test_batch['eeg'].cuda())
                test_predicts.extend(predict.cpu().tolist())
                test_reals.extend(test_batch['label'].tolist())

        test_predicts = np.array(test_predicts)
        test_reals = np.array(test_reals)

        print('Test scores')
        print(precision_recall_fscore_support(test_reals, (test_predicts > 0).astype(int).reshape(-1)))

        val_predicts = []
        val_reals = []
        for val_batch in test_excl_dataloader_F:
            with torch.no_grad():
                predict = model2(val_batch['eeg'].cuda())
                val_predicts.extend(predict.cpu().tolist())
                val_reals.extend(val_batch['label'].tolist())

        val_predicts = np.array(val_predicts)
        val_reals = np.array(val_reals)

        print('Val scores')
        print(precision_recall_fscore_support(val_reals, (val_predicts > 0).astype(int).reshape(-1)))
    
        val_predicts = []
        val_reals = []
        for val_batch in test_excl_demo_dataloader_F:
            with torch.no_grad():
                predict = model2(val_batch['eeg'].cuda())
                val_predicts.extend(predict.cpu().tolist())
                val_reals.extend(val_batch['label'].tolist())

        val_predicts = np.array(val_predicts)
        val_reals = np.array(val_reals)

        print('Val demo scores')
        print(precision_recall_fscore_support(val_reals, (val_predicts > 0).astype(int).reshape(-1)))
    

Epoch:  0


/tmp/ipykernel_3339018/700437237.py:23: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3277.)
  'eeg': torch.from_numpy(data).float().T,


In [ ]:

for epoch in range(7):
    print('Epoch: ', epoch)
    for batch in train_excl_dataloader_F:
        optimizer2.zero_grad()
        predict = model2(batch['eeg'].cuda()).view(-1)
        
        loss = loss_fnc(predict, batch['label'].cuda())
        loss.backward()
        
        optimizer2.step()
    
    if epoch % 5 == 0:
        test_predicts = []
        test_reals = []
        for test_batch in test_dataloader_F:
            with torch.no_grad():
                predict = model2(test_batch['eeg'].cuda())
                test_predicts.extend(predict.cpu().tolist())
                test_reals.extend(test_batch['label'].tolist())

        test_predicts = np.array(test_predicts)
        test_reals = np.array(test_reals)

        print('Test scores')
        print(precision_recall_fscore_support(test_reals, (test_predicts > 0).astype(int).reshape(-1)))

        val_predicts = []
        val_reals = []
        for val_batch in test_excl_dataloader_F:
            with torch.no_grad():
                predict = model2(val_batch['eeg'].cuda())
                val_predicts.extend(predict.cpu().tolist())
                val_reals.extend(val_batch['label'].tolist())

        val_predicts = np.array(val_predicts)
        val_reals = np.array(val_reals)

        print('Val scores')
        print(precision_recall_fscore_support(val_reals, (val_predicts > 0).astype(int).reshape(-1)))

In [ ]:
val_predicts = []
val_reals = []
for val_batch in test_excl_demo_dataloader_F:
    with torch.no_grad():
        predict = model2(val_batch['eeg'].cuda())
        val_predicts.extend(predict.cpu().tolist())
        val_reals.extend(val_batch['label'].tolist())

val_predicts = np.array(val_predicts)
val_reals = np.array(val_reals)

print('Val demo scores')
print(precision_recall_fscore_support(val_reals, (val_predicts > 0.5).astype(int).reshape(-1)))


In [ ]:
predicted_results = {
    'second_model_finetune_metrics_on_val': precision_recall_fscore_support(val_reals, (val_predicts > 0).astype(int).reshape(-1))
}

In [45]:
val_reals

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.])

In [46]:
predicted_results

{'second_model_finetune_metrics_on_val': (array([0.42857143, 0.53030303]),
  array([0.08823529, 0.8974359 ]),
  array([0.14634146, 0.66666667]),
  array([34, 39]))}

In [52]:
# import json
# with open('../results/torch.results_{}.inner.our_data.json'.format(exclude_user[0]), 'w') as f:
#     json.dump(predicted_results, f)

TypeError: Object of type ndarray is not JSON serializable